In [ ]:
## 算法 keras 自定义loss  Matlab data
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os
import tensorflow_probability as tfp
import scipy.io as scio

# DeepNN topology
layers=[5,20,20,20,20,20,20,6]
# layers=[2,50,50,20,1]
tf_optimizer = tf.keras.optimizers.Adam(
   learning_rate=0.002, beta_1=0.99, epsilon=1e-1)
# tf_optimizer = tf.keras.optimizers.Adam()

xmax=0
xmin=-1.5
ymax=1.0
ymin=-1.0

# sampling points
NH=100000
x1=tf.random.uniform([NH],xmin,xmax,dtype=tf.float32)
x2=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
x3=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
x4=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
x5=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)

class mymodel():
    def __init__(self,layers,optimizer,x1,x2,x3,x4,x5):
        self.u_model=tf.keras.Sequential()
        self.u_model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
        self.u_model.add(tf.keras.layers.Lambda(lambda X: X))
        for width in layers[1:7]:
            self.u_model.add(tf.keras.layers.Dense(
            width, activation=tf.nn.tanh,
            kernel_initializer='glorot_normal'))
        
        self.u_model.add(tf.keras.layers.Dense(
            layers[7],
            kernel_initializer='glorot_normal'))
        
        # Computing the sizes of weights/biases for future decomposition
        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(layers):
            if i != 1:
                self.sizes_w.append(int(width * layers[1]))
                self.sizes_b.append(int(width if i != 0 else layers[1]))
        
        self.optimizer=optimizer
        self.dtype=tf.float32

    # Defining custom loss
    def __loss(self):
        Xnode=tf.constant([-1,0,0,0,0],shape=(1,5),dtype=tf.float32)
        u_pred=self.u_model(Xnode)
        u_pred0=u_pred[:,0]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x1)
            tape.watch(x2)
            tape.watch(x3)
            tape.watch(x4)
            tape.watch(x5)
            X_f = tf.stack([x1,x2,x3,x4,x5], axis=1)
            u = self.u_model(X_f)
            u0=u[:,0]
            u1=u[:,1]
            u2=u[:,2]
            u3=u[:,3]
            u4=u[:,4]
            u5=u[:,5]
    
        u_x1=tape.gradient(u0, x1)
        #u_x1=tf.reduce_mean(u_x1,axis=1)
        u_x1=u_x1+2*x1+2
        u_x2=tape.gradient(u0, x2)
        u_x2=u_x2+2*x2
        u_x3=tape.gradient(u0, x3)
        u_x3=u_x3+2*x3
        u_x4=tape.gradient(u0, x4)
        u_x4=u_x4+2*x4
        u_x5=tape.gradient(u0, x5)
        u_x5=u_x5+2*x5
        
        b1=-2*(x1**3-x1)-(x2+x3+x4+x5)
        b2=-x2+2*(x1**3-x1)
        b3=-x3+2*(x1**3-x1)
        b4=-x4+2*(x1**3-x1)
        b5=-x5+2*(x1**3-x1)
        delta=0.001
        
        return tf.reduce_mean(tf.square(b1+0.5*u_x1-u1),axis=0) * 1 + \
            tf.reduce_mean(tf.square(b2+0.5*u_x2-u2),axis=0) * 1 + \
            tf.reduce_mean(tf.square(b3+0.5*u_x3-u3),axis=0) * 1 + \
            tf.reduce_mean(tf.square(b4+0.5*u_x4-u4),axis=0) * 1 + \
            tf.reduce_mean(tf.square(b5+0.5*u_x5-u5),axis=0) * 1 + \
            tf.reduce_mean((u_x1*u1+u_x2*u2+u_x3*u3+u_x4*u4+u_x5*u5)**2/((u_x1**2+u_x2**2+u_x3**2+u_x4**2+u_x5**2)*(u1**2+u2**2+u3**2+u4**2+u5**2)+delta),axis=0) * 1 + \
            tf.reduce_mean(tf.square(u_pred0),axis=0) * 0.1

    def __grad(self):
        with tf.GradientTape() as tape:
            loss_value = self.__loss()
        return loss_value, tape.gradient(loss_value, self.__wrap_training_variables())

    def __wrap_training_variables(self):
        var = self.u_model.trainable_variables
        return var

    def summary(self):
        return self.u_model.summary()

    # The training function
    def fit(self, tf_epochs=5000):
    # Creating the tensors
    #X_u = tf.convert_to_tensor(X_u, dtype=self.dtype)
    #u = tf.convert_to_tensor(u, dtype=self.dtype)
    #print(self.__wrap_training_variables())

        LOSS=np.zeros([1,tf_epochs])
        for epoch in range(tf_epochs):
            # Optimization step
            loss_value, grads = self.__grad()
            self.optimizer.apply_gradients(zip(grads, self.__wrap_training_variables()))
            LOSS[0,epoch]=loss_value.numpy()
            print('epoch, loss_value:', epoch, loss_value)

        scio.savemat('LOSS.mat',{'LOSS':LOSS})

    def predict(self, X_star):
        u_star = self.u_model(X_star)
        # f_star = self.H_model()
        return u_star#, f_star


model=mymodel(layers, tf_optimizer, x1, x2, x3, x4, x5)

# checkpoint_save_path="./checkpoint/mnist.ckpt"
# if os.path.exists(checkpoint_save_path + '.index'):
#   print('-------------------load the model---------------------')
#   model.load_weights(checkpoint_save_path)

# cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
#                         save_weights_only=True,
#                         save_best_only=True)

history=model.fit(tf_epochs=100000)

model.summary()

# Getting the model predictions, from the same (x,t) that the predictions were previously gotten from
x01=tf.constant([-1.5,-1.5,-1.5,-1,-1,-1,0,0,0],dtype=tf.float32)
x02=tf.constant([-0.8,0,0.8,-0.8,0,0.8,-0.8,0,0.8],dtype=tf.float32)
x03=tf.constant([-0.8,0,0.8,-0.8,0,0.8,-0.8,0,0.8],dtype=tf.float32)
x04=tf.constant([-0.8,0,0.8,-0.8,0,0.8,-0.8,0,0.8],dtype=tf.float32)
x05=tf.constant([-0.8,0,0.8,-0.8,0,0.8,-0.8,0,0.8],dtype=tf.float32)
X_star = tf.stack([x01,x02,x03,x04,x05], axis=1)
print('X_star:\n ', X_star)
u_pred= model.predict(X_star)
print('u_pred:\n ', u_pred)


In [ ]:
## computing Stest
dataFile = "x1test"  
data = scio.loadmat(dataFile)
x1test = data['x1test']
dataFile = "x2test"  
data = scio.loadmat(dataFile)
x2test = data['x2test']
dataFile = "x3test"  
data = scio.loadmat(dataFile)
x3test = data['x3test']
dataFile = "x4test"  
data = scio.loadmat(dataFile)
x4test = data['x4test']
dataFile = "x5test"  
data = scio.loadmat(dataFile)
x5test = data['x5test']
#print(x1test)

x1test =tf.reduce_sum(x1test, axis=0)
x2test =tf.reduce_sum(x2test, axis=0)
x3test =tf.reduce_sum(x3test, axis=0)
x4test =tf.reduce_sum(x4test, axis=0)
x5test =tf.reduce_sum(x5test, axis=0)
#print(x1test)
X_star = tf.stack([x1test,x2test,x3test,x4test,x5test], axis=1)
Stest= model.predict(X_star)
scio.savemat('Stest.mat',{'Stest':Stest.numpy()})

In [ ]:
## finding the location of minimal quasi-potential on the boundary x1=-0.5
xm1=-0.5
xm2=0.5
xm3=0
xm4=0
xm5=0.5
eta=0.01

for epoch in range(1000):
    x01=tf.constant(xm1,shape=(1,),dtype=tf.float32)
    x02=tf.constant(xm2,shape=(1,),dtype=tf.float32)
    x03=tf.constant(xm3,shape=(1,),dtype=tf.float32)
    x04=tf.constant(xm4,shape=(1,),dtype=tf.float32)
    x05=tf.constant(xm5,shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        tape.watch(x03)
        tape.watch(x04)
        tape.watch(x05)
        # Packing together the inputs
        X_f = tf.stack([x01,x02,x03,x04,x05], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
    # Getting the derivative
    u_x2 = tape.gradient(u0, x02)
    u_x3 = tape.gradient(u0, x03)
    u_x4 = tape.gradient(u0, x04)
    u_x5 = tape.gradient(u0, x05)
    u_x2 = u_x2+2*x02
    u_x3 = u_x3+2*x03
    u_x4 = u_x4+2*x04
    u_x5 = u_x5+2*x05
    # Letting the tape go
    del tape
    
    p2=u_x2.numpy()
    p3=u_x3.numpy()
    p4=u_x4.numpy()
    p5=u_x5.numpy()
    
    xm2=xm2-eta*p2
    xm3=xm3-eta*p3
    xm4=xm4-eta*p4
    xm5=xm5-eta*p5
    print(epoch,xm1,xm2,xm3,xm4,xm5)

In [ ]:
## computing MPEP and prefactor characteristic boundary
detH_bar=128.0
detH_star=64.0
lambda_star=1
pi=3.141592653

h=0.001
nT=10000
x1=np.zeros(nT,)
x2=np.zeros(nT,)
x3=np.zeros(nT,)
x4=np.zeros(nT,)
x5=np.zeros(nT,)
divL=np.zeros(nT,)
#normbx=np.zeros(nT,)
x1[0]=-0.05
x2[0]=0
x3[0]=0
x4[0]=0
x5[0]=0

for epoch in range(nT-1):
    x01=tf.constant(x1[epoch],shape=(1,),dtype=tf.float32)
    x02=tf.constant(x2[epoch],shape=(1,),dtype=tf.float32)
    x03=tf.constant(x3[epoch],shape=(1,),dtype=tf.float32)
    x04=tf.constant(x4[epoch],shape=(1,),dtype=tf.float32)
    x05=tf.constant(x5[epoch],shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        tape.watch(x03)
        tape.watch(x04)
        tape.watch(x05)
        # Packing together the inputs
        X_f = tf.stack([x01,x02,x03,x04,x05], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
        u1=u[:,1]
        u2=u[:,2]
        u3=u[:,3]
        u4=u[:,4]
        u5=u[:,5]
    # Getting the derivative
    u_x1 = tape.gradient(u0, x01)
    u_x2 = tape.gradient(u0, x02)
    u_x3 = tape.gradient(u0, x03)
    u_x4 = tape.gradient(u0, x04)
    u_x5 = tape.gradient(u0, x05)
    u_x1 = u_x1+2*x01+2
    u_x2 = u_x2+2*x02
    u_x3 = u_x3+2*x03
    u_x4 = u_x4+2*x04
    u_x5 = u_x5+2*x05
    l1_x1 = tape.gradient(u1, x01)
    l2_x2 = tape.gradient(u2, x02)
    l3_x3 = tape.gradient(u3, x03)
    l4_x4 = tape.gradient(u4, x04)
    l5_x5 = tape.gradient(u5, x05)
    # Letting the tape go
    del tape
    divL[epoch]=l1_x1.numpy()+l2_x2.numpy()+l3_x3.numpy()+l4_x4.numpy()+l5_x5.numpy()
    
    if ((x1[epoch]+1.0)**2+(x2[epoch])**2+(x3[epoch])**2+(x4[epoch])**2+(x5[epoch])**2)**0.5<0.01:
        break
    
    p1=u_x1.numpy()
    p2=u_x2.numpy()
    p3=u_x3.numpy()
    p4=u_x4.numpy()
    p5=u_x5.numpy()
    bx1=-2*(x1[epoch]**3-x1[epoch])-(x2[epoch]+x3[epoch]+x4[epoch]+x5[epoch])
    bx2=-x2[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx3=-x3[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx4=-x4[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx5=-x5[epoch]+2*(x1[epoch]**3-x1[epoch])
    normb=(bx1**2+bx2**2+bx3**2+bx4**2+bx5**2)**0.5
    # normbx[epoch]=normb
    x1[epoch+1]=x1[epoch] -h * (bx1 + p1)/normb
    x2[epoch+1]=x2[epoch] -h * (bx2 + p2)/normb
    x3[epoch+1]=x3[epoch] -h * (bx3 + p3)/normb
    x4[epoch+1]=x4[epoch] -h * (bx4 + p4)/normb
    x5[epoch+1]=x5[epoch] -h * (bx5 + p5)/normb

print('epoch:',epoch)
MPEP=tf.stack([x1[0:epoch+1],x2[0:epoch+1],x3[0:epoch+1],x4[0:epoch+1],x5[0:epoch+1]], axis=1)
scio.savemat('MPEP.mat',{'MPEP':MPEP.numpy()})
print(MPEP)
integ=np.sum(divL[0:epoch+1])*h
prefactor=pi/lambda_star*(detH_star/detH_bar)**0.5*np.exp(integ)
print('integ:',integ)
print('prefactor:',prefactor)

In [ ]:
## computing MPEP and prefactor non-characteristic boundary
detH_bar=128.0
deth2_star=16.0
miu2_star=0.75
pi=3.141592653

h=0.001
nT=10000
x1=np.zeros(nT,)
x2=np.zeros(nT,)
x3=np.zeros(nT,)
x4=np.zeros(nT,)
x5=np.zeros(nT,)
divL=np.zeros(nT,)
#normbx=np.zeros(nT,)
x1[0]=-0.5
x2[0]=0.0005
x3[0]=-0.0009
x4[0]=-0.0001
x5[0]=0

for epoch in range(nT-1):
    x01=tf.constant(x1[epoch],shape=(1,),dtype=tf.float32)
    x02=tf.constant(x2[epoch],shape=(1,),dtype=tf.float32)
    x03=tf.constant(x3[epoch],shape=(1,),dtype=tf.float32)
    x04=tf.constant(x4[epoch],shape=(1,),dtype=tf.float32)
    x05=tf.constant(x5[epoch],shape=(1,),dtype=tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x01)
        tape.watch(x02)
        tape.watch(x03)
        tape.watch(x04)
        tape.watch(x05)
        # Packing together the inputs
        X_f = tf.stack([x01,x02,x03,x04,x05], axis=1)
        # Getting the prediction
        u = model.predict(X_f)
        u0=u[:,0]
        u1=u[:,1]
        u2=u[:,2]
        u3=u[:,3]
        u4=u[:,4]
        u5=u[:,5]
    # Getting the derivative
    print('V:',u0+(x01+1)**2+x02**2+x03**2+x04**2+x05**2)
    u_x1 = tape.gradient(u0, x01)
    u_x2 = tape.gradient(u0, x02)
    u_x3 = tape.gradient(u0, x03)
    u_x4 = tape.gradient(u0, x04)
    u_x5 = tape.gradient(u0, x05)
    u_x1 = u_x1+2*x01+2
    u_x2 = u_x2+2*x02
    u_x3 = u_x3+2*x03
    u_x4 = u_x4+2*x04
    u_x5 = u_x5+2*x05
    l1_x1 = tape.gradient(u1, x01)
    l2_x2 = tape.gradient(u2, x02)
    l3_x3 = tape.gradient(u3, x03)
    l4_x4 = tape.gradient(u4, x04)
    l5_x5 = tape.gradient(u5, x05)
    # Letting the tape go
    del tape
    divL[epoch]=l1_x1.numpy()+l2_x2.numpy()+l3_x3.numpy()+l4_x4.numpy()+l5_x5.numpy()
    
    if ((x1[epoch]+1.0)**2+(x2[epoch])**2+(x3[epoch])**2+(x4[epoch])**2+(x5[epoch])**2)**0.5<0.01:
        break
    
    p1=u_x1.numpy()
    p2=u_x2.numpy()
    p3=u_x3.numpy()
    p4=u_x4.numpy()
    p5=u_x5.numpy()
    bx1=-2*(x1[epoch]**3-x1[epoch])-(x2[epoch]+x3[epoch]+x4[epoch]+x5[epoch])
    bx2=-x2[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx3=-x3[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx4=-x4[epoch]+2*(x1[epoch]**3-x1[epoch])
    bx5=-x5[epoch]+2*(x1[epoch]**3-x1[epoch])
    normb=(bx1**2+bx2**2+bx3**2+bx4**2+bx5**2)**0.5
    # normbx[epoch]=normb
    x1[epoch+1]=x1[epoch] -h * (bx1 + p1)/normb
    x2[epoch+1]=x2[epoch] -h * (bx2 + p2)/normb
    x3[epoch+1]=x3[epoch] -h * (bx3 + p3)/normb
    x4[epoch+1]=x4[epoch] -h * (bx4 + p4)/normb
    x5[epoch+1]=x5[epoch] -h * (bx5 + p5)/normb

print('epoch:',epoch)
MPEP=tf.stack([x1[0:epoch+1],x2[0:epoch+1],x3[0:epoch+1],x4[0:epoch+1],x5[0:epoch+1]], axis=1)
scio.savemat('MPEP.mat',{'MPEP':MPEP.numpy()})
print(MPEP)
integ=np.sum(divL[0:epoch+1])*h
prefactor=1/miu2_star*(2*pi*deth2_star/detH_bar)**0.5*np.exp(integ)
print('integ:',integ)
print('prefactor:',prefactor)

In [ ]:
## Approximation error
NH=100000
xt1=tf.random.uniform([NH],xmin,xmax,dtype=tf.float32)
xt2=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
xt3=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
xt4=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
xt5=tf.random.uniform([NH],ymin,ymax,dtype=tf.float32)
X_f = tf.stack([xt1,xt2,xt3,xt4,xt5], axis=1)

Vtrue=(1-xt1**2)**2+xt2**2+xt3**2+xt4**2+xt5**2
l1true=-(xt2+xt3+xt4+xt5)
l2true=2*(xt1**3-xt1)
l3true=2*(xt1**3-xt1)
l4true=2*(xt1**3-xt1)
l5true=2*(xt1**3-xt1)

u = model.predict(X_f)
u0=u[:,0]+(xt1+1)**2+xt2**2+xt3**2+xt4**2+xt5**2
u1=u[:,1]
u2=u[:,2]
u3=u[:,3]
u4=u[:,4]
u5=u[:,5]

eV=tf.reduce_max((u0-Vtrue)**2,axis=0)/tf.reduce_max((Vtrue)**2,axis=0)
el=tf.reduce_max((u1-l1true)**2+(u2-l2true)**2+(u3-l3true)**2+(u4-l4true)**2+(u5-l5true)**2,axis=0)/tf.reduce_max((l1true)**2+(l2true)**2+(l3true)**2+(l4true)**2+(l5true)**2,axis=0)
print(eV,el)